In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [ ]:
cat_sse_plusTranch.to_csv('cat_sse_plusTranch.csv', index = False)

cat_sse_plusTranch.replace('-', np.nan, inplace=True)

df1 = cat_sse_plusTranch[['depth', 'mag', 'strike', 'dip', 'length', 'width',
       'slip', 'duration']]

# A (miss 7%): age
# alphaS (miss 15%): shallow slab dip measured between 0 and 125 km depth
# ZT (miss 0): trench depth
# Tchannel (miss 50%): subduction channel sediment thickness (potentially subject to large variations) 
# Phi: thermal parameter, product of A with the vertical component of trench-normal subduction velocity 
# M56_vs: subduction velocity, accounting for upper plate deformation
# Vup1 (miss 0): upper plate velocity
# Vsub1 (miss 0): subducting plate velocity
# Vt (miss 0): trench velocity
# M56_vc: convergence velocity 
col_to_corr = ['A', 'alphaS', 'ZT', 'Tchannel', 'Phi', 'M56_vs', 'M56_vc' ,'Vup1', 'Vsub1', 'Vt1']

df2 = cat_sse_plusTranch[col_to_corr]
# If df1 and df2 have different columns and you want *all pairwise* correlations:
results_corr = pd.DataFrame(index=['depth', 'mag', 'strike', 'dip', 'length', 'width',
       'slip', 'duration'], columns=col_to_corr)
results_pv = pd.DataFrame(index=['depth', 'mag', 'strike', 'dip', 'length', 'width',
       'slip', 'duration'], columns=col_to_corr)

for col1 in df1.columns:
    for col2 in df2.columns:
        s1 = df1[col1]
        s2 = df2[col2]
        mask = ~np.isnan(s1) & ~np.isnan(s2)
        s1 = s1[mask]
        s2 = s2[mask]
        # Compute correlation with dropped NaNs
        corr, p_value = pearsonr(s1, s2)
        results_corr.loc[col1, col2] = corr
        results_pv.loc[col1, col2] = p_value

# Flatten the DataFrame
filtered_corr = results_corr.stack().astype(float)

filtered_corr = filtered_corr[(filtered_corr > 0.2) | (filtered_corr < -0.2)]
#filtered_pv = filtered_pv[(filtered_corr > 0.2) | (filtered_corr < -0.2)]

# Optional: sort values
filtered_corr = filtered_corr.sort_values(ascending=False)

# Show result
print(filtered_corr)
